<a href="https://colab.research.google.com/github/AtifQureshi110/BERT/blob/main/Q%7CA_baby_is_locked%F0%9F%98%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import BertForQuestionAnswering, BertTokenizer

In [88]:
# Conversational Question Answering dataset, released by Stanford NLP in JSON format
coqa_df =pd.read_json('http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json')
coqa_df['data'][:5]

0    {'source': 'wikipedia', 'id': '3zotghdk5ibi9ce...
1    {'source': 'cnn', 'id': '3wj1oxy92agboo5nlq4r7...
2    {'source': 'gutenberg', 'id': '3bdcf01ogxu7zdn...
3    {'source': 'cnn', 'id': '3ewijtffvo7wwchw6rtya...
4    {'source': 'gutenberg', 'id': '3urfvvm165iantk...
Name: data, dtype: object

In [89]:
coqa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7199 entries, 0 to 7198
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   version  7199 non-null   int64 
 1   data     7199 non-null   object
dtypes: int64(1), object(1)
memory usage: 112.6+ KB


In [90]:
del coqa_df["version"]

In [91]:
print(coqa_df.data)

0       {'source': 'wikipedia', 'id': '3zotghdk5ibi9ce...
1       {'source': 'cnn', 'id': '3wj1oxy92agboo5nlq4r7...
2       {'source': 'gutenberg', 'id': '3bdcf01ogxu7zdn...
3       {'source': 'cnn', 'id': '3ewijtffvo7wwchw6rtya...
4       {'source': 'gutenberg', 'id': '3urfvvm165iantk...
                              ...                        
7194    {'source': 'gutenberg', 'id': '34j10vatjfyw0ao...
7195    {'source': 'cnn', 'id': '3vj40nv2qinjocrcy7k4z...
7196    {'source': 'race', 'id': '3rjsc4xj10uw0to3vq0v...
7197    {'source': 'wikipedia', 'id': '3gs6s824sqxty8v...
7198    {'source': 'cnn', 'id': '31qnsg6a5rtt5m7pens7x...
Name: data, Length: 7199, dtype: object


In [92]:
coqa_df.data[2]

{'source': 'gutenberg',
 'id': '3bdcf01ogxu7zdn9vlrbf2rqzwplyf',
 'filename': 'data/gutenberg/txt/Zane Grey___Riders of the Purple Sage.txt/CHAPTER VII_78c077ef5e268383edbec1f1c9d644b1423f889d258d95ff055aa92',
 'story': 'CHAPTER VII. THE DAUGHTER OF WITHERSTEEN \n\n"Lassiter, will you be my rider?" Jane had asked him. \n\n"I reckon so," he had replied. \n\nFew as the words were, Jane knew how infinitely much they implied. She wanted him to take charge of her cattle and horse and ranges, and save them if that were possible. Yet, though she could not have spoken aloud all she meant, she was perfectly honest with herself. Whatever the price to be paid, she must keep Lassiter close to her; she must shield from him the man who had led Milly Erne to Cottonwoods. In her fear she so controlled her mind that she did not whisper this Mormon\'s name to her own soul, she did not even think it. Besides, beyond this thing she regarded as a sacred obligation thrust upon her, was the need of a helper,

# understanding  with json data and it's exraction processes

In [93]:
for i,j in coqa_df.data[0].items():
  print(i, ":", j,"\n")

source : wikipedia 

id : 3zotghdk5ibi9cex97fepx7jetpso7 

filename : Vatican_Library.txt 

story : The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. 

The Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. 

In March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to be made availabl

In [94]:
for i in coqa_df.data[0]:
  print(i)

source
id
filename
story
questions
answers
name


In [95]:
for question in coqa_df.data[0]["questions"]:
    print(question['input_text'])

When was the Vat formally opened?
what is the library for?
for what subjects?
and?
what was started in 2014?
how do scholars divide the library?
how many?
what is the official name of the Vat?
where is it?
how many printed books does it contain?
when were the Secret Archives moved from the rest of the library?
how many items are in this secret collection?
Can anyone use this library?
what must be requested to view?
what must be requested in person or by mail?
of what books?
What is the Vat the library of?
How many books survived the Pre Lateran period?
what is the point of the project started in 2014?
what will this allow?


In [96]:
story_list = []

# Iterate through the "story" string
for char in coqa_df.data[3]["story"]:
    story_list.append(char)

# Join the characters in the list to form the complete story
complete_story = ''.join(story_list)

# Print the complete story
print(complete_story)


(CNN) -- The longest-running holiday special still has a very shiny nose. 

"Rudolph the Red-Nosed Reindeer" premiered on television December 6, 1964, and is now one of the holiday season's perennial favorites. The story of the reindeer who saves Christmas is beloved among children and adults alike. 

The Rankin-Bass animated film production company used Japanese puppets and stop motion to tell the tale, bolstered by a soundtrack featuring Burl Ives' rendition of the theme song. 

In the story, Santa's reindeer Donner and his wife have a son, Rudolph, who has the distinction of a nose that glows. He runs away after being made to feel an outcast and links up with an elf who dreams of becoming a dentist and an adventurer seeking silver and gold. 

After ending up on the Island of Misfit Toys and wandering for a while, Rudolph goes on to save his loved ones from the Abominable Snow Monster and guides Santa through a blizzard that threatens to ruin Christmas. 

In 2006, the New York Times 

In [97]:
for asnwer in coqa_df.data[0]["answers"]:
    print(asnwer['input_text'])

It was formally established in 1475
research
history, and law
philosophy, science and theology
a  project
into periods
five
The Vatican Apostolic Library
in Vatican City
1.1 million
at the beginning of the 17th century;
150,000
anyone who can document their qualifications and research needs.
unknown
Photocopies
only books published between 1801 and 1990
the Holy See
a handful of volumes
digitising manuscripts
them to be viewed online.


In [98]:
question_list=[]
answer_list=[]
for question, answer in zip(coqa_df.data[0]["questions"], coqa_df.data[0]["answers"]):
    question_list.append(question['input_text'])
    answer_list.append(answer['input_text'])

In [99]:
df = pd.DataFrame({"Questions":question_list,"Answers":answer_list})

In [100]:
df

,Questions,Answers
0,When was the Vat formally opened?,It was formally established in 1475
1,what is the library for?,research
2,for what subjects?,"history, and law"
3,and?,"philosophy, science and theology"
4,what was started in 2014?,a project
5,how do scholars divide the library?,into periods
6,how many?,five
7,what is the official name of the Vat?,The Vatican Apostolic Library
8,where is it?,in Vatican City
9,how many printed books does it contain?,1.1 million


In [101]:
question_list = []
answer_list = []

# Iterate through records at index 0 and index 2
for i in [0, 2]:
    for question, answer in zip(coqa_df.data[i]["questions"], coqa_df.data[i]["answers"]):
        question_list.append(question['input_text'])
        answer_list.append(answer['input_text'])


In [102]:
df = pd.DataFrame({"Questions":question_list,"Answers":answer_list})

In [103]:
df

,Questions,Answers
0,When was the Vat formally opened?,It was formally established in 1475
1,what is the library for?,research
2,for what subjects?,"history, and law"
3,and?,"philosophy, science and theology"
4,what was started in 2014?,a project
5,how do scholars divide the library?,into periods
6,how many?,five
7,what is the official name of the Vat?,The Vatican Apostolic Library
8,where is it?,in Vatican City
9,how many printed books does it contain?,1.1 million


In [105]:
question_list = []
answer_list = []

# Define the range of indices you want to iterate through (e.g., 0 to 2)
start_index = 0
end_index = 7198  # You can adjust this to the desired end index

# Iterate through the specified range of indices
for i in range(start_index, end_index + 1):
    for question, answer in zip(coqa_df.data[i]["questions"], coqa_df.data[i]["answers"]):
        question_list.append(question['input_text'])
        answer_list.append(answer['input_text'])


In [106]:
df = pd.DataFrame({"Questions":question_list,"Answers":answer_list})

In [107]:
df.to_csv("coqa_df.csv",index=False)

In [108]:
pd.read_csv("/content/coqa_df.csv")

,Questions,Answers
0,When was the Vat formally opened?,It was formally established in 1475
1,what is the library for?,research
2,for what subjects?,"history, and law"
3,and?,"philosophy, science and theology"
4,what was started in 2014?,a project
...,...,...
108642,Who was a sub?,Xabi Alonso
108643,Was it his first game this year?,Yes
108644,What position did the team reach?,third
108645,Who was ahead of them?,Barca.


# targeted information has kept successsfully 😎.

In [112]:
question_list = []
answer_list = []
story_list = []

# Define the range of indices you want to iterate through (e.g., 0 to 2)
start_index = 0
end_index = 7198   # You can adjust this to the desired end index

# Iterate through the specified range of indices
for i in range(start_index, end_index + 1):
    for question, answer in zip(coqa_df.data[i]["questions"], coqa_df.data[i]["answers"]):
        question_list.append(question['input_text'])
        answer_list.append(answer['input_text'])

        # Append the story (assuming 'story' is a string)
        story_list.append(coqa_df.data[i]["story"])


In [113]:
df = pd.DataFrame({"Questions":question_list,"Answers":answer_list,"Story":story_list})

In [114]:
df

,Questions,Answers,Story
0,When was the Vat formally opened?,It was formally established in 1475,"The Vatican Apostolic Library (), more commonl..."
1,what is the library for?,research,"The Vatican Apostolic Library (), more commonl..."
2,for what subjects?,"history, and law","The Vatican Apostolic Library (), more commonl..."
3,and?,"philosophy, science and theology","The Vatican Apostolic Library (), more commonl..."
4,what was started in 2014?,a project,"The Vatican Apostolic Library (), more commonl..."
...,...,...,...
108642,Who was a sub?,Xabi Alonso,(CNN) -- Cristiano Ronaldo provided the perfec...
108643,Was it his first game this year?,Yes,(CNN) -- Cristiano Ronaldo provided the perfec...
108644,What position did the team reach?,third,(CNN) -- Cristiano Ronaldo provided the perfec...
108645,Who was ahead of them?,Barca.,(CNN) -- Cristiano Ronaldo provided the perfec...


In [115]:
df.to_csv("coqa_df.csv",index=False)

In [116]:
df = pd.read_csv("/content/coqa_df.csv")

In [117]:
df.head(2)

,Questions,Answers,Story
0,When was the Vat formally opened?,It was formally established in 1475,"The Vatican Apostolic Library (), more commonl..."
1,what is the library for?,research,"The Vatican Apostolic Library (), more commonl..."


## another way to do it.

In [109]:
#required columns in our dataframe
cols = ["text","question","answer"]
#list of lists to create our dataframe
comp_list = []
for index, row in coqa_df.iterrows():
    for i in range(len(row["data"]["questions"])):
        temp_list = []
        temp_list.append(row["data"]["story"])
        temp_list.append(row["data"]["questions"][i]["input_text"])
        temp_list.append(row["data"]["answers"][i]["input_text"])
        comp_list.append(temp_list)
new_df = pd.DataFrame(comp_list, columns=cols)

In [127]:
new_df[0:22]

,text,question,answer
0,"The Vatican Apostolic Library (), more commonl...",When was the Vat formally opened?,It was formally established in 1475
1,"The Vatican Apostolic Library (), more commonl...",what is the library for?,research
2,"The Vatican Apostolic Library (), more commonl...",for what subjects?,"history, and law"
3,"The Vatican Apostolic Library (), more commonl...",and?,"philosophy, science and theology"
4,"The Vatican Apostolic Library (), more commonl...",what was started in 2014?,a project
5,"The Vatican Apostolic Library (), more commonl...",how do scholars divide the library?,into periods
6,"The Vatican Apostolic Library (), more commonl...",how many?,five
7,"The Vatican Apostolic Library (), more commonl...",what is the official name of the Vat?,The Vatican Apostolic Library
8,"The Vatican Apostolic Library (), more commonl...",where is it?,in Vatican City
9,"The Vatican Apostolic Library (), more commonl...",how many printed books does it contain?,1.1 million


In [129]:
#saving the dataframe to csv file for further loading
new_df.to_csv("coqa_df_data.csv", index=False)

In [131]:
print("Number of question and answers: ", len(new_df))

Number of question and answers:  108647


# Model

In [109]:
model_id = 'bert-large-uncased-whole-word-masking-finetuned-squad'

In [110]:
model = BertForQuestionAnswering.from_pretrained (model_id)
tokenizer = BertTokenizer.from_pretrained (model_id)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [118]:
random_num = np.random.randint(0,len(df))
print("random number has selected: ",random_num)
question = df["Questions"][random_num]
text = df["Story"][random_num]

random number has selected:  24241


In [119]:
random_num

24241

In [120]:
print("random question according to random number is: ",question)

random question according to random number is:  How did he become stranded?


In [121]:
print("random text according to random number is: ",text)

random text according to random number is:  (CNN) -- A man stranded after his car plunged down a steep embankment in the Angeles National Forest survived for six days by eating leaves and drinking water from a creek, authorities said Friday. 

David J. Lavau, 67, of Lake Hughes, California, was found in a ravine a week after losing control of his car on a rural road and plunging 500 feet down an embankment into heavy brush, according to a report by the California Highway Patrol. 

Lavau, who is partially disabled, told authorities that he spent the first night in his car. 

"The next morning, he exited his vehicle and observed another vehicle adjacent to his own with a deceased male driver behind the wheel," the report said. "The deceased appeared to have been there for some time." 

Authorities say they have not identified the dead driver. 

The case began to unfold on September 23, when Lavau failed to return home. 

Lavau's family began searching for him when he failed to return hom

In [122]:
# the encode function of BertTokenizer converts words into numerical values so machine could them prorperly when data feed to it.
input_ids = tokenizer.encode(question, text)
print("The input has a total of {} tokens.".format(len(input_ids)))

The input has a total of 353 tokens.


In [123]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)
#  convert_ids_to_tokens is a function  of BertTokenzer which is used to convert token_IDs back to human-readable or textual form
for token, id in zip(tokens, input_ids):
    # {:8}{:8,} --> it used to create the space b/w token and words
    print('{:8}{:8,}'.format(token,id))

[CLS]        101
how        2,129
did        2,106
he         2,002
become     2,468
stranded  15,577
?          1,029
[SEP]        102
(          1,006
cnn       13,229
)          1,007
-          1,011
-          1,011
a          1,037
man        2,158
stranded  15,577
after      2,044
his        2,010
car        2,482
plunged   16,687
down       2,091
a          1,037
steep      9,561
embankment  22,756
in         1,999
the        1,996
angeles    3,349
national   2,120
forest     3,224
survived   5,175
for        2,005
six        2,416
days       2,420
by         2,011
eating     5,983
leaves     3,727
and        1,998
drinking   5,948
water      2,300
from       2,013
a          1,037
creek      3,636
,          1,010
authorities   4,614
said       2,056
friday     5,958
.          1,012
david      2,585
j          1,046
.          1,012
lava      13,697
##u        2,226
,          1,010
67         6,163
,          1,010
of         1,997
lake       2,697
hughes     8,099
,        

In [124]:
#first occurence of [SEP] token
sep_idx = input_ids.index(tokenizer.sep_token_id)
print("SEP token index: ", sep_idx)

SEP token index:  7


In [125]:
#number of tokens in segment A (question) - this will be one more than the sep_idx as the index in Python starts from 0
num_seg_a = sep_idx+1
print("Number of tokens in segment A: ", num_seg_a)

Number of tokens in segment A:  8


In [126]:
#number of tokens in segment B (text) mean the text is conssit of 279 tokens from those anwer will find.
num_seg_b = len(input_ids) - num_seg_a
print("Number of tokens in segment B: ", num_seg_b)

Number of tokens in segment B:  345


In [127]:
#creating the segment ids
segment_ids = [0]*num_seg_a + [1]*num_seg_b
print("Combination of question tokens and text tokens.0 shows thetokens of questions and 1 show the tokens text. \n",segment_ids,'\n')
print("This is combine lenght of question tokens and text tokens. ", len(segment_ids),'\n')
print("""This is lenght of of input_id which got when i encode the (text) and (question) by
(BertTokenizer instance called tokenizer).""",len(input_ids))
#making sure that every input token has a segment id
assert len(segment_ids) == len(input_ids)

Combination of question tokens and text tokens.0 shows thetokens of questions and 1 show the tokens text. 
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [128]:
#Let’s now feed this to our model.
#token input_ids to represent the input and token segment_ids to differentiate our segments - question and text
output = model(torch.tensor([input_ids]),  token_type_ids=torch.tensor([segment_ids]))
print("""1stly--> it creates a tensor of input_ids (which has token of 288) using the PyTorch library \n
2ndly--> it goes segment_ids (it combination of question tokens and text token) and creates the tensors using pytroch library\n""")
print("The result of output is full of numeric values so, it's better not to print b/c it design for only machine not for human")

1stly--> it creates a tensor of input_ids (which has token of 288) using the PyTorch library 

2ndly--> it goes segment_ids (it combination of question tokens and text token) and creates the tensors using pytroch library

The result of output is full of numeric values so, it's better not to print b/c it design for only machine not for human


In [129]:
#tokens with highest start and end scores
answer_start = torch.argmax(output.start_logits)
answer_end = torch.argmax(output.end_logits)
print("these are variables basically build the boundary arounthe answer.")
if answer_end >= answer_start: # IF CONDITION IS TRUE it print the answer
    # remember that end token is always greater than the start token.
    answer = " ".join(tokens[answer_start:answer_end+1])
else:
    print("I am unable to find the answer to this question. Can you please ask another question?")

print("\nQuestion:\n{}".format(question.capitalize()))
print("\nAnswer:\n{}.".format(answer.capitalize()))

these are variables basically build the boundary arounthe answer.

Question:
How did he become stranded?

Answer:
After his car plunged down a steep embankment.


- Here all  main conceptual code is kept in function just to know how code work work in function as well

In [164]:
def question_answer(question, text):

    #tokenize question and text as a pair
    input_ids = tokenizer.encode(question, text)

    #string version of tokenized ids
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    #segment IDs
    #first occurence of [SEP] token
    sep_idx = input_ids.index(tokenizer.sep_token_id)
    #number of tokens in segment A (question)
    num_seg_a = sep_idx+1
    #number of tokens in segment B (text)
    num_seg_b = len(input_ids) - num_seg_a

    #list of 0s and 1s for segment embeddings
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)

    #model output using input_ids and segment_ids
    output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))

    #reconstructing the answer
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    if answer_end >= answer_start:
        answer = tokens[answer_start]
        for i in range(answer_start+1, answer_end+1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]

    if answer.startswith("[CLS]"):
        answer = "Unable to find the answer to your question."

    print("\nPredicted answer:\n{}".format(answer.capitalize()))

In [165]:
text = """New York (CNN) -- More than 80 Michael Jackson collectibles -- including the late pop star's famous rhinestone-studded glove from a 1983 performance -- were auctioned off Saturday, reaping a total $2 million. Profits from the auction at the Hard Rock Cafe in New York's Times Square crushed pre-sale expectations of only $120,000 in sales. The highly prized memorabilia, which included items spanning the many stages of Jackson's career, came from more than 30 fans, associates and family members, who contacted Julien's Auctions to sell their gifts and mementos of the singer. Jackson's flashy glove was the big-ticket item of the night, fetching $420,000 from a buyer in Hong Kong, China. Jackson wore the glove at a 1983 performance during \"Motown 25,\" an NBC special where he debuted his revolutionary moonwalk. Fellow Motown star Walter \"Clyde\" Orange of the Commodores, who also performed in the special 26 years ago, said he asked for Jackson's autograph at the time, but Jackson gave him the glove instead. "The legacy that [Jackson] left behind is bigger than life for me,\" Orange said. \"I hope that through that glove people can see what he was trying to say in his music and what he said in his music.\" Orange said he plans to give a portion of the proceeds to charity. Hoffman Ma, who bought the glove on behalf of Ponte 16 Resort in Macau, paid a 25 percent buyer's premium, which was tacked onto all final sales over $50,000. Winners of items less than $50,000 paid a 20 percent premium."""
question = "Where was the Auction held?"
question_answer(question, text)
#original answer from the dataset
print("Original answer:\n", df.loc[df["Questions"] == question]["Answers"].values[0])


Predicted answer:
Hard rock cafe in new york ' s times square
Original answer:
 Hard Rock Cafe


- Checking by answer of question from the data frame as well.

In [166]:
question_to_find = "Where was the Auction held?"
matching_row = df[df['Questions'] == question_to_find]

In [167]:
matching_row

,Questions,Answers,Story
20,Where was the Auction held?,Hard Rock Cafe,New York (CNN) -- More than 80 Michael Jackson...


- Here dealing with user input

In [169]:
df['Story'][39]

'CHAPTER VII. THE DAUGHTER OF WITHERSTEEN \n\n"Lassiter, will you be my rider?" Jane had asked him. \n\n"I reckon so," he had replied. \n\nFew as the words were, Jane knew how infinitely much they implied. She wanted him to take charge of her cattle and horse and ranges, and save them if that were possible. Yet, though she could not have spoken aloud all she meant, she was perfectly honest with herself. Whatever the price to be paid, she must keep Lassiter close to her; she must shield from him the man who had led Milly Erne to Cottonwoods. In her fear she so controlled her mind that she did not whisper this Mormon\'s name to her own soul, she did not even think it. Besides, beyond this thing she regarded as a sacred obligation thrust upon her, was the need of a helper, of a friend, of a champion in this critical time. If she could rule this gun-man, as Venters had called him, if she could even keep him from shedding blood, what strategy to play his flame and his presence against the g

In [171]:
text = input("Please enter your text: \n")
question = input("\nPlease enter your question: \n")
while True:
    question_answer(question, text)

    flag = True
    flag_N = False

    while flag:
        response = input("\nDo you want to ask another question based on this text (Y/N)? ")
        if response[0] == "Y":
            question = input("\nPlease enter your question: \n")
            flag = False
        elif response[0] == "N":
            print("\nBye!")
            flag = False
            flag_N = True

    if flag_N == True:
        break

Please enter your text: 
CHAPTER VII. THE DAUGHTER OF WITHERSTEEN   "Lassiter, will you be my rider?" Jane had asked him.   "I reckon so," he had replied.   Few as the words were, Jane knew how infinitely much they implied. She wanted him to take charge of her cattle and horse and ranges, and save them if that were possible. Yet, though she could not have spoken aloud all she meant, she was perfectly honest with herself. Whatever the price to be paid, she must keep Lassiter close to her; she must shield from him the man who had led Milly Erne to Cottonwoods. In her fear she so controlled her mind that she did not whisper this Mormon's name to her own soul, she did not even think it. Besides, beyond this thing she regarded as a sacred obligation thrust upon her, was the need of a helper, of a friend, of a champion in this critical time. If she could rule this gun-man, as Venters had called him, if she could even keep him from shedding blood, what strategy to play his flame and his prese

In [160]:
df[39:45]

,Questions,Answers,Story
39,Who did that affect?,Tull and his men,"CHAPTER VII. THE DAUGHTER OF WITHERSTEEN \n\n""..."
40,Did Jane think she could control Lassiter?,No,"CHAPTER VII. THE DAUGHTER OF WITHERSTEEN \n\n""..."
41,Who is Bells?,One of her safe racers,"CHAPTER VII. THE DAUGHTER OF WITHERSTEEN \n\n""..."
42,How did he get his name?,because of the way he struck his iron shoes on...,"CHAPTER VII. THE DAUGHTER OF WITHERSTEEN \n\n""..."
43,Was Lassiter impressed with the horse?,Yes,"CHAPTER VII. THE DAUGHTER OF WITHERSTEEN \n\n""..."
44,Did he want to take him for himself?,Yes,"CHAPTER VII. THE DAUGHTER OF WITHERSTEEN \n\n""..."


In [85]:
questions_to_find = ["When was the Vat formally opened?", "what is the library for?","for what subjects?"]
matching_rows = df[df['Questions'].isin(questions_to_find)]

In [86]:
matching_rows

,Questions,Answers,Story
0,When was the Vat formally opened?,It was formally established in 1475,"The Vatican Apostolic Library (), more commonl..."
1,what is the library for?,research,"The Vatican Apostolic Library (), more commonl..."
2,for what subjects?,"history, and law","The Vatican Apostolic Library (), more commonl..."
